In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (
    KFold,
    RepeatedStratifiedKFold,
    StratifiedKFold,
    GridSearchCV,
    cross_val_predict
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from typing import Dict, Any

# ==============================================================================
# FASE 1: CONFIGURAÇÃO E CARGA DE DADOS
# ==============================================================================

def print_header(title: str) -> None:
    """Imprime um cabeçalho formatado no console."""
    print("\n" + "="*80)
    print(f"| {title.center(76)} |")
    print("="*80)

# --- Configurações Globais ---
DATASET_URL = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
COLUMN_NAMES = [
    'Nº de Gestações', 'Concentração de Glicose', 'Pressão Arterial (mmHg)',
    'Espessura da Pele (mm)', 'Nível de Insulina (mu U/ml)', 'Índice de Massa Corporal',
    'Histórico Familiar de Diabetes', 'Idade (anos)', 'Classe'
]
TARGET_COLUMN = 'Classe'

PARAM_GRID = [
    {
        'svm__C': [0.01, 0.1, 1, 10, 100],
        'svm__kernel': ['linear']
    },
    {
        'svm__C': [0.01, 0.1, 1, 10, 100],
        'svm__kernel': ['rbf'],
        'svm__gamma': ['scale', 0.01, 0.001]
    }
]

CV_STRATEGIES = {
    "K-Fold (10 splits)": KFold(n_splits=10, shuffle=True, random_state=42),
    "Repeated Stratified K-Fold (20x2)": RepeatedStratifiedKFold(n_splits=20, n_repeats=2, random_state=42)
}

print_header("FASE 1: CONFIGURAÇÃO E CARGA DE DADOS")
try:
    df = pd.read_csv(DATASET_URL, header=None, names=COLUMN_NAMES)
    print(" -> Dataset 'Pima Indians Diabetes' carregado com sucesso.")
    print(f" -> Dimensões iniciais: {df.shape[0]} amostras e {df.shape[1]} colunas.")

    print("\n -> Verificando e tratando valores ausentes (representados por '0')...")
    cols_com_zeros_ausentes = [
        'Concentração de Glicose', 'Pressão Arterial (mmHg)',
        'Espessura da Pele (mm)', 'Nível de Insulina (mu U/ml)', 'Índice de Massa Corporal'
    ]
    df[cols_com_zeros_ausentes] = df[cols_com_zeros_ausentes].replace(0, np.nan)
    print(f" -> Contagem de valores nulos após substituição:\n{df.isnull().sum()}")

    X = df.drop(TARGET_COLUMN, axis=1)
    y = df[TARGET_COLUMN]

except Exception as e:
    print(f" -> Falha ao carregar o dataset: {e}")
    X, y = None, None

# ==============================================================================
# FASE 2: ANÁLISE EXPLORATÓRIA DE DADOS (EDA)
# ==============================================================================
def plot_eda_graphs(X: pd.DataFrame, y: pd.Series) -> None:
    print_header("FASE 2: ANÁLISE EXPLORATÓRIA DE DADOS (EDA)")

    print("\n--- Distribuição das Features por Classe ---")
    df_plot = pd.concat([X, y.rename(TARGET_COLUMN)], axis=1)
    n_features = len(X.columns)
    n_cols = 2
    n_rows = (n_features + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, n_rows * 4))
    axes = axes.flatten()
    for i, col in enumerate(X.columns):
        sns.histplot(data=df_plot, x=col, hue=TARGET_COLUMN, kde=True, ax=axes[i], palette='viridis', multiple='stack')
        axes[i].set_title(f'Distribuição de "{col}"')

    if n_features < len(axes):
        for i in range(n_features, len(axes)):
            fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

    print("\n--- Heatmap de Correlação entre as Features ---")
    plt.figure(figsize=(10, 8))
    sns.heatmap(X.corr(), annot=True, cmap='viridis', fmt='.2f')
    plt.title('Correlação entre as Variáveis Preditivas')
    plt.show()


# ==============================================================================
# FASE 3: METODOLOGIA E EXECUÇÃO DA ANÁLISE COMPARATIVA
# ==============================================================================

def run_comparative_analysis(X: pd.DataFrame, y: pd.Series) -> Dict[str, Any]:
    print_header("FASE 3: EXECUÇÃO DA ANÁLISE COMPARATIVA")

    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('svm', SVC(random_state=42, probability=True))
    ])

    results_summary = {}

    for name, cv_strategy in CV_STRATEGIES.items():
        print(f"\n--- Analisando com a estratégia: {name} ---")

        grid_search = GridSearchCV(
            pipeline, PARAM_GRID, cv=cv_strategy, scoring='accuracy', n_jobs=-1, verbose=0
        )

        print(" -> Buscando melhores hiperparâmetros com GridSearchCV...")
        grid_search.fit(X, y)
        print(" -> Busca concluída.")

        best_score_mean = grid_search.best_score_
        best_param_index = np.argmin(grid_search.cv_results_['rank_test_score'])
        best_score_std = grid_search.cv_results_['std_test_score'][best_param_index]

        print(f"\n   - Acurácia Média: {best_score_mean:.4f} (±{best_score_std:.4f})")
        print(f"   - Melhores Hiperparâmetros: {grid_search.best_params_}")

        print(" -> Gerando predições com cross_val_predict para uma matriz de confusão mais precisa...")

        best_params_svm = {k.replace('svm__', ''): v for k, v in grid_search.best_params_.items()}
        best_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('svm', SVC(**best_params_svm, random_state=42, probability=True))
        ])

        if isinstance(cv_strategy, RepeatedStratifiedKFold):
            n_splits = cv_strategy.get_n_splits()
            cv_for_predict = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
            print(f"    (Aviso) Usando StratifiedKFold com {n_splits} splits para gerar a matriz de confusão.")
        else:
            cv_for_predict = cv_strategy

        y_pred = cross_val_predict(best_pipeline, X, y, cv=cv_for_predict, n_jobs=-1)

        cm = confusion_matrix(y, y_pred)

        results_summary[name] = {
            "acuracia_media": best_score_mean,
            "desvio_padrao": best_score_std,
            "melhores_parametros": grid_search.best_params_,
            "matriz_confusao": cm,
            "cv_results": pd.DataFrame(grid_search.cv_results_)
        }

        fig, ax = plt.subplots(figsize=(6, 5))
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Não Diabético', 'Diabético'])
        disp.plot(cmap=plt.cm.Blues, ax=ax)
        ax.set_title(f'Matriz de Confusão Representativa para\n{name}')
        plt.show()

    return results_summary

# ==============================================================================
# FASE 4: ANÁLISE COMPARATIVA FINAL DOS RESULTADOS
# ==============================================================================
def plot_final_comparison(results: Dict[str, Any]) -> None:
    print_header("FASE 4: ANÁLISE COMPARATIVA FINAL DOS RESULTADOS")

    names = list(results.keys())
    means = [res['acuracia_media'] for res in results.values()]
    stds = [res['desvio_padrao'] for res in results.values()]

    plt.figure(figsize=(8, 6))
    bars = plt.bar(names, means, yerr=stds, capsize=5, color=sns.color_palette("viridis", len(names)))
    plt.ylabel('Acurácia Média')
    plt.title('Comparação da Acurácia Média entre Estratégias de CV')

    min_y = min(means) - max(stds) - 0.02
    max_y = max(means) + max(stds) + 0.02
    plt.ylim(max(0, min_y), min(1, max_y))

    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.4f}', ha='center', va='bottom', color='black', fontweight='bold')
    plt.show()

    all_scores_data = []
    for name, result in results.items():
        cv_results_df = result['cv_results']
        best_param_mask = cv_results_df['params'].apply(lambda param_dict: param_dict == result['melhores_parametros'])

        split_score_columns = [col for col in cv_results_df.columns if 'split' in col and 'test_score' in col]

        scores_for_best_params = cv_results_df.loc[best_param_mask, split_score_columns].values.flatten()

        for score in scores_for_best_params:
            all_scores_data.append({'Estratégia': name, 'Acurácia': score})

    scores_df = pd.DataFrame(all_scores_data)

    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Estratégia', y='Acurácia', data=scores_df, palette='viridis')
    sns.stripplot(x='Estratégia', y='Acurácia', data=scores_df, color='black', alpha=0.3, jitter=0.2)
    plt.title('Distribuição dos Scores de Acurácia nos Folds')
    plt.ylabel('Acurácia')
    plt.xlabel('Estratégia de Validação Cruzada')
    plt.grid(axis='y', linestyle='--')
    plt.show()

# --- Execução Principal ---
if __name__ == "__main__":
    if 'X' in locals() and X is not None:
        # FASE 2
        plot_eda_graphs(X, y)

        # FASE 3
        final_results = run_comparative_analysis(X, y)

        # FASE 4
        if final_results:
            plot_final_comparison(final_results)
    else:
        print(" -> Variáveis X e y não foram definidas. A execução foi interrompida.")